# 如何計算出累進費率？

假設你今天用了 $540$ 度的電，我們想要把度數根據以下的表格：

| 度數 | 平日價格 | 假日價格 |
| ---- | -------- | -------- |
| 100  | 0.25     | 0.50     |
| 200  | 0.5      | 0.75     |
| 300  | 0.75     | 1        |
| 400  | 1        | 1.25     |
| 500  | 1.25     | 1.5      |

轉換成這樣的輸出：

    本日度數：540
    平日費率：305
    假日費率：440

怎麼做？

## 如果不借助電腦，要怎麼拆？

用了 $540$ 度的電，是不是就等於：

- 前 $100$ 度的電，按 100 度電的標準計算。
- 接下來 $200$ 度的電，按 200 度電的標準計算。
- 最後 $240$ ($540-100-200$) 度的電，按 300 度電的標準計算？

## 邏輯上會長什麼樣子？

詳細拆解邏輯。令 $x$ 為使用度數 ($540$)，
$r$ 是這個區間的值（$100, 200, 300$)：

- $x$ 剛開始是 $540$，大於 $100$。所以記下「第一個區間吃了 100 度的電，」
  然後把 $x$ 減掉 $100$，繼續下個流程。
- $x$ 現在是 $440$，大於 $200$，所以記下「第二個區間吃了 200 度的電，」
  然後把 $x$ 減掉 $200$，繼續下個流程。
- $x$ 現在是 $240$，小於 $300$，所以記下「第三個區間吃了 300 度的電。」

嘿，寫成程式不就是：



In [3]:
x = 540
separated = []

if x < 100:
  separated += [x]
else:
  separated += [100]
  x -= 100

if x < 200:
  separated += [x]
else:
  separated += [200]
  x -= 200

if x < 300:
  separated += [x]
else:
  separated += [300]

print(separated)

[100, 200, 240]


確實很接近了。但我們可以寫得更好。

## 搭配 $for$ 迴圈拉出累計費率

我們可以看到每段程式碼都長得像這樣：

```py
if x < 費率:
    separated += [x]
else:
    separated += [費率]
    x -= 費率
```

那我們可不可以用 $for$ 迴圈取代複製貼上呢？當然可以：

In [4]:
x = 540
separated = []

# 為什麼是這些數字呢？看看最上面的表格～
for 費率 in [100, 200, 300, 400, 500]:
    if x < 費率:
        separated += [x]
    else:
        separated += [費率]
        x -= 費率

print(separated)

[100, 200, 240, 240, 240]


但是我們看到「當 $x$ 已經算完之後，它還是不停地把剩下
的不正確數字填進去。」考慮到 $x < 費率$ 已經是 **結尾** 的
情況了，我們便直接 `break` 結束掉剩下不必要的計算。

In [5]:
x = 540
separated = []

for 費率 in [100, 200, 300, 400, 500]:
    if x < 費率:
        separated += [x]
        break
    else:
        separated += [費率]
        x -= 費率

print(separated)

[100, 200, 240]


接下來我們還需要把它乘以費率。因此我們再寫一次 for 迴圈，
把這些區間和費率做相乘：

In [6]:
平日價格表 = [0.25, 0.5, 0.75, 1, 1.25]
假日價格表 = [0.5, 0.75, 1, 1.25, 1.5]

平日P = 0
假日P = 0

# 目前的區間 = 1, 2, 3, 4, …
for 目前的區間 in range(0, len(separated)):
    平日P += separated[目前的區間] * 平日價格表[目前的區間]
    假日P += separated[目前的區間] * 假日價格表[目前的區間]

print(平日P, 假日P)

305.0 440.0


接近完美了！另外上面的 range 可以改用 `enumerate` 代替：

In [7]:
平日價格表 = [0.25, 0.5, 0.75, 1, 1.25]
假日價格表 = [0.5, 0.75, 1, 1.25, 1.5]

平日P = 0
假日P = 0

# 目前的區間 = 1, 2, 3, 4, …
for 目前的區間, 區間度數 in enumerate(separated):
    平日P += 區間度數 * 平日價格表[目前的區間]
    假日P += 區間度數 * 假日價格表[目前的區間]

print(平日P, 假日P)

305.0 440.0


整合一下，答案就是：

In [11]:
inputs = int(input("請輸入金額："))
x = inputs
separated = []

for 費率 in [100, 200, 300, 400, 500]:
    if x < 費率:
        separated += [x]
        break
    else:
        separated += [費率]
        x -= 費率

平日價格表 = [0.25, 0.5, 0.75, 1, 1.25]
假日價格表 = [0.5, 0.75, 1, 1.25, 1.5]

平日P = 0
假日P = 0

# 目前的區間 = 1, 2, 3, 4, …
for 目前的區間, 區間度數 in enumerate(separated):
    平日P += 區間度數 * 平日價格表[目前的區間]
    假日P += 區間度數 * 假日價格表[目前的區間]

print(f"本日度數：{inputs}")
print(f"平日費率：{平日P:.0f}")
print(f"假日度數：{假日P:.0f}")

本日度數：540
平日費率：305
假日度數：440
